# Lab #1 (Solving systems of linear equations)
### By Stakhov K.V. [8.1212]

### 1. Solve the given SLE using `numpy.linalg.solve`

#### First we define utility functions for loading matrices & vectors from files:

In [1]:
import numpy as np
import numpy.linalg as lin
import time

def load_file(name, for_each_line):
    ret = []
    try:
        with open("../input/lab1-matrices/" + name) as file:
            for line in file:
                ret.append(for_each_line(line))
        return ret
    except:
        raise Exception(f"File '{name}' doesn't exist.")

def load_coef_matrix(name):
    return np.array(
        load_file(
            name,
            lambda line: [int(n) for n in line.split()]
        )
    )

def load_ordinate_vector(name):
    return np.array(
        load_file(
            name,
            lambda line: int(line)
        )
    )

#### A function for measuring time spent executing arbitrary functions:

In [2]:
def measure_elapsed_time(action, *args):
    start = time.time_ns()
    action(*args)
    elapsed_ms = (time.time_ns() - start) / 1000000
    return elapsed_ms

#### A function to print SLE solutions:

In [3]:
def print_solution(a, b, x):
    print(f"SLE solution (A[{a.shape[0]}x{a.shape[1]}], b[{b.size}]): ")
    for n in range(x.size):
        print(f"x{n + 1} = {x[n]}")

#### Then, we use the functions defined above to load the SLE's coefficient matrix and ordinate vector from the given files:

In [4]:
coef_file = input("Enter SLE coefficient matrix filename: ")
ord_file = input("Enter SLE ordinate vector filename: ")

a = load_coef_matrix(coef_file)
b = load_ordinate_vector(ord_file)
print("A:\n", a, "\nB:\n", b)

Enter SLE coefficient matrix filename:  a.mat
Enter SLE ordinate vector filename:  b.vec


A:
 [[ 2  3  1  0 -1]
 [ 1  0 -2  1  0]
 [-1  1  3 -2  1]
 [ 0  2  1  0  2]
 [ 1  0  0 -1  1]] 
B:
 [ 4 -3  2  1  0]


#### And solve the system using `numpy.linalg.solve`:

In [5]:
def solve_numpy(a, b):
    return lin.solve(a, b)

print_solution(a, b, solve_numpy(a, b))

SLE solution (A[5x5], b[5]): 
x1 = 3.692307692307694
x2 = -2.7692307692307714
x3 = 5.461538461538465
x4 = 4.230769230769234
x5 = 0.5384615384615392


### 2. Solve the given SLE using Cramer's Rule and `numpy.linalg.det`

In [6]:
def solve_cramer(a, b):
    n = b.size # Number of variables in the SLE
    x = [] # Solution vector x
    det_a = lin.det(a) # Determinant of the coefficient matrix

    # Create matrices A_n by sequentially replacing the n'th column by the ordinate vector B
    # Then calculate det(A_n) and divide it by det(A) to get the n'th SLE solution
    for i in range(n):
        an = a.copy()
        for k in range(n):
            an[k][i] = b[k]
        x.append(lin.det(an) / det_a)

    return np.array(x)

print_solution(a, b, solve_cramer(a, b))

SLE solution (A[5x5], b[5]): 
x1 = 3.6923076923076925
x2 = -2.7692307692307687
x3 = 5.4615384615384635
x4 = 4.2307692307692335
x5 = 0.538461538461539


**As we can see, two implemented methods produced same results.**

### 3. Build a timesheet for different numbers of rows in a square matrix. I.e., compare computing time of both methods for a size with 10, 20, 30, ..., 500 rows.

In [7]:
from numpy.random import default_rng

elapsed = []
rng = default_rng(1) # Set seed for values to be the same on every generation
methods = [solve_numpy, solve_cramer]

min_size = 10
max_size = 500 + min_size

tab = "    "
print("N", *[method.__name__ for method in methods], "delta% (second/first)", sep = tab)

# Compute the SLE solution for a randomly generated matrix A[NxN] and vector B[N]
# using two methods defined above and measure execution time. 
for size in range(min_size, max_size, min_size):
    print(size, end = tab)
    method_elapsed = []
    for method in methods:
        a = rng.random((size, size))
        b = rng.random((size))
        elapsed_ms = measure_elapsed_time(method, a, b)
        method_elapsed.append(elapsed_ms)
        print(f"{elapsed_ms} ms", sep = "", end = tab)
        elapsed.append(method_elapsed)
    delta = 100 * (method_elapsed[1] / method_elapsed[0])
    print(f"{delta}%")

N    solve_numpy    solve_cramer    delta% (second/first)
10    0.066855 ms    0.194493 ms    290.9176576172313%
20    0.415801 ms    1.425131 ms    342.74352394534884%
30    0.130934 ms    4.166907 ms    3182.4484091221534%
40    0.245727 ms    6.232398 ms    2536.3098072250914%
50    0.228774 ms    9.150139 ms    3999.6411305480515%
60    0.196074 ms    14.032296 ms    7156.632699899018%
70    0.303802 ms    19.788322 ms    6513.558831080769%
80    0.316807 ms    25.741884 ms    8125.4151581246615%
90    0.367016 ms    34.96496 ms    9526.821718944133%
100    0.470928 ms    42.775466 ms    9083.228434070601%
110    0.454719 ms    55.600348 ms    12227.408135573838%
120    0.536837 ms    67.122615 ms    12503.351110299773%
130    0.621423 ms    83.333841 ms    13410.163608363388%
140    0.589556 ms    100.026482 ms    16966.40895860614%
150    0.646627 ms    117.113203 ms    18111.400080726602%
160    0.763706 ms    146.155905 ms    19137.718572330188%
170    0.814739 ms    166.630456

**As we can see, the built-in `numpy.linalg.solve` is much faster than our implementation of Cramer's rule. The bigger the matrix size, the slower our version gets, starting at 2.9x and up to 1006x slower in this case.**